In [9]:
import tensorflow as tf
import tensorflow_datasets as tfds
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pathlib
import os

In [8]:
builder = tfds.ImageFolder("dataset")
print(builder.info)

UnicodeDecodeError: 'utf-8' codec can't decode byte 0xd7 in position 61: invalid continuation byte

In [5]:
img_height = 180
img_width = 180
batch_size = 32

In [11]:
train_ds = tf.keras.utils.image_dataset_from_directory(
    "dataset",
    validation_split = 0.2,
    subset = "training",
    seed = 420,
    image_size = (img_height, img_width),
    batch_size = batch_size
)

AttributeError: 'list' object has no attribute 'with_suffix'